In [192]:
# packages
from lxml import etree
import pandas as pd
from tqdm import tqdm
import os



In [193]:
#setup
#path = 'OneDrive_2_19-04-2023'
xml_file= 'part2/20230531-213702-ALL CONTRACTS.xml'
excel_crm = 'part2/templates/2023-05-31 CRM_Contract_Conversion_Template_v0.5.xlsx'
excel_nmd = 'part2/templates/2023-05-31 NMD_Contract_Conversion_Template_v0.5.xlsx'
sheet_name1 = 'Price Program - Static'
sheet_name2 = 'Price Prog - Static-Tiered'
sheet_name_rem = 'Contract Header'

pricing_crm = 'part2/pricing/2023-05-31 CRMSalesDeal-RK20230526.csv'
pricing_nmd = 'part2/pricing/2023-05-31 NMD Sales Deals 2023.05.24.csv'
#remove = '20230530-173628-log.csv'


In [194]:
# Opening excel and xml file
tree = etree.parse(f'{xml_file}')
root=tree.getroot()

excel_crm1 = pd.read_excel(f'{excel_crm}', header = 2, sheet_name=f'{sheet_name1}')
excel_crm2 = pd.read_excel(f'{excel_crm}', header = 2, sheet_name=f'{sheet_name2}')
excel_crm_rem = pd.read_excel(f'{excel_crm}', header = 2, sheet_name=f'{sheet_name_rem}')
excel_nmd1 = pd.read_excel(f'{excel_nmd}', header = 2, sheet_name=f'{sheet_name1}')
excel_nmd2 = pd.read_excel(f'{excel_nmd}', header = 2, sheet_name=f'{sheet_name2}')
excel_nmd_rem = pd.read_excel(f'{excel_nmd}', header = 2, sheet_name=f'{sheet_name_rem}')
pricing_crm = pd.read_csv(f'{pricing_crm}')
pricing_nmd = pd.read_csv(f'{pricing_nmd}')
#remove = pd.read_csv(f'{remove}', header=None)
#excel_nmd = pd.read_excel(f'{path}/{excel_nmd}', header = 2, sheet_name=f'{sheet_name}')
#load = pd.read_excel(f'{path}/{load_file}')
#df_log = pd.read_csv(f'{path}/logs.csv', header=None)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(

In [195]:
# # contracts to be removed
# dropping rows
excel_crm_rem.dropna(axis = 0, how = 'all', inplace = True)
excel_nmd_rem.dropna(axis = 0, how = 'all', inplace = True)
excel_crm_rem = excel_crm_rem[1:-1]
excel_nmd_rem = excel_nmd_rem[1:-1]


In [196]:
# data types
excel_crm_rem['Customer Contract ID'] = pd.to_numeric(excel_crm_rem['Customer Contract ID'])
excel_crm_rem['Customer Contract ID'] = excel_crm_rem['Customer Contract ID'].astype(float)
crm_remove = list(excel_crm_rem['Customer Contract ID'].drop_duplicates(keep='last'))
excel_nmd_rem['Customer Contract ID'] = pd.to_numeric(excel_nmd_rem['Customer Contract ID'])
excel_nmd_rem['Customer Contract ID'] = excel_nmd_rem['Customer Contract ID'].astype(float)
nmd_remove = list(excel_nmd_rem['Customer Contract ID'].drop_duplicates(keep='last'))




In [9]:
# list of to be removed contracts
remove = list(remove.loc[:,0].str[8:13].drop_duplicates(keep='last'))

In [282]:
# # # removing products from log from excel
# remove['ID'] = remove['ID'].astype('string')
# remove = list(remove['ID'].drop_duplicates(keep='last'))



In [197]:
# dropping rows
excel_nmd2.dropna(axis = 0, how = 'all', inplace = True)
excel_nmd2.dropna(axis = 1, how = 'all', inplace = True)
excel_crm1.dropna(axis = 0, how = 'all', inplace = True)
excel_crm2.dropna(axis = 0, how = 'all', inplace = True)
excel_nmd2 = excel_nmd2[1:-1]
excel_nmd1 = excel_nmd1[1:]
excel_crm1 = excel_crm1[1:]
excel_crm2 = excel_crm2[1:]

In [198]:
# merge two sheets
excel_crm = pd.concat([excel_crm1, excel_crm2])
excel_nmd = pd.concat([excel_nmd1, excel_nmd2])
excel_crm.dropna(axis = 0, how = 'all', inplace = True)
#excel_nmd.dropna(axis = 0, how = 'all', inplace = True)

In [199]:
# reseting indexes
excel_nmd = excel_nmd.reset_index(drop=True)
excel_crm = excel_crm.reset_index(drop=True)
excel_crm = excel_crm[:-1]

In [201]:
# data types
excel_crm['Customer Contract ID'] = pd.to_numeric(excel_crm['Customer Contract ID'], errors='coerce')
excel_nmd['Customer Contract ID'] = pd.to_numeric(excel_nmd['Customer Contract ID'], errors='coerce')

In [202]:
# # removing contracts that are not in contract header
for index in tqdm(excel_crm.index):
    if excel_crm['Customer Contract ID'][index] not in crm_remove:
        excel_crm.drop([index], axis=0, inplace=True)


100%|██████████| 550/550 [00:00<00:00, 53440.53it/s]


In [203]:
# # removing contracts that are not in contract header
for index in tqdm(excel_nmd.index):
    if excel_nmd['Customer Contract ID'][index] not in nmd_remove:
        excel_nmd.drop([index], axis=0, inplace=True)

100%|██████████| 2165/2165 [00:00<00:00, 52621.12it/s]


In [204]:
# splitting lists of sales deals into separate rows
excel_nmd['Sales Deal #']= excel_nmd['Sales Deal #'].astype(str)
excel_nmd['Sales Deal #']  = excel_nmd['Sales Deal #'].apply(lambda x: x.split(','))
excel_nmd = excel_nmd.explode('Sales Deal #')
excel_nmd = excel_nmd.reset_index(drop=True)
# splitting lists of sales deals into separate rows
excel_crm['Sales Deal #']= excel_crm['Sales Deal #'].astype(str)
excel_crm['Sales Deal #']  = excel_crm['Sales Deal #'].apply(lambda x: x.split(','))
excel_crm = excel_crm.explode('Sales Deal #')
excel_crm = excel_crm.reset_index(drop=True)

In [131]:
# mergeing two excels
#excel_both = pd.concat([excel_crm, excel_nmd])


In [133]:
# # splitting lists of sales deals into separate rows
# excel_both['Sales Deal #']= excel_both['Sales Deal #'].astype(str)
# excel_both['Sales Deal #']  = excel_both['Sales Deal #'].apply(lambda x: x.split(','))
# excel_both = excel_both.explode('Sales Deal #')
# excel_both = excel_both.reset_index(drop=True)


In [ ]:
#######PRICING########

In [205]:
# merging crm and nmd pricing
pricing_nmd.rename(columns = {'Sales Deal':'SalesDeal', 'Sales Deal Type': 'DealType', 'Sales Org': 'SalesOrg', 
'Dist.Channel':'DChan', 'Division':'Divi', 'Valid From':'ValidFr', 'Valid To':'ValidTo', 'Currency':'Curr',
'External Description':'ExDesc',  'Promotion Number':'PromotionNo','PH Level':'PLevel'}, inplace = True)
#pricing_both = pd.concat([pricing_crm, pricing_nmd])
#pricing_both = pricing_both.reset_index(drop=True)

In [135]:
# data types
# pricing_both['PLevel']= pricing_both['PLevel'].astype('Int64')
# pricing_both['Product No'] = pricing_both['PLevel'].astype(str)+'_'+pricing_both['Material/PH'].astype(str)

In [206]:
# creating product id
pricing_nmd['PLevel']= pricing_nmd['PLevel'].astype('Int64')
pricing_nmd['Product No'] = pricing_nmd['PLevel'].astype(str)+'_'+pricing_nmd['Material/PH'].astype(str)

pricing_crm['PLevel']= pricing_crm['PLevel'].astype('Int64')
pricing_crm['Product No'] = pricing_crm['PLevel'].astype(str)+'_'+pricing_crm['Material/PH'].astype(str)

In [207]:
# without removing from log
pricing_crm_after_check = pricing_crm
pricing_nmd_after_check = pricing_nmd

In [ ]:
### removing from log

In [73]:
# condition to minimize the time needed to remove products
condition_nmd = (pricing_nmd['PLevel'] == 5) & (pricing_nmd['Material/PH'].str.len() == 3)
pricing_nmd_check= pricing_nmd.loc[condition_nmd]
pricing_nmd__wothout_check= pricing_nmd.loc[-condition_nmd]

condition_crm = (pricing_crm['PLevel'] == 5) & (pricing_crm['Material/PH'].str.len() == 3)
pricing_crm_check= pricing_crm.loc[condition_crm]
pricing_crm__wothout_check= pricing_crm.loc[-condition_crm]

In [192]:
# reset index
pricing_crm_check = pricing_crm_check.reset_index(drop=True)
pricing_nmd_check = pricing_nmd_check.reset_index(drop=True)

In [144]:
#condition
# condition = (pricing_both['PLevel'] == 5) & (pricing_both['Material/PH'].str.len() == 3)
# pricing_both_check= pricing_both.loc[condition]
# pricing_both__wothout_check= pricing_both.loc[-condition]


In [145]:
# reset index
#pricing_both_check = pricing_both_check.reset_index(drop=True)

In [147]:
# # removing deleted products
# for index in tqdm(pricing_both_check.index):
#     if pricing_both_check['Product No'][index] in remove:
#         pricing_both_check.drop([index], axis=0, inplace=True)


100%|██████████| 165874/165874 [34:11<00:00, 80.84it/s]  


In [194]:
# # removing deleted products
for index in tqdm(pricing_nmd_check.index):
    if pricing_nmd_check['Product No'][index] in remove:
        pricing_nmd_check.drop([index], axis=0, inplace=True)

0it [00:00, ?it/s]


In [198]:
# # removing deleted products
for index in tqdm(pricing_crm_check.index):
    if pricing_crm_check['Product No'][index] in remove:
        pricing_crm_check.drop([index], axis=0, inplace=True)

100%|██████████| 165874/165874 [27:47<00:00, 99.50it/s]  


In [199]:
# merging pricing and sales
pricing_crm_after_check = pd.concat([pricing_crm_check, pricing_crm__wothout_check])
pricing_crm_after_check = pricing_crm_after_check.reset_index(drop=True)
pricing_crm_after_check.to_csv('pricing crm.csv')

pricing_nmd_after_check = pd.concat([pricing_nmd_check, pricing_nmd__wothout_check])
pricing_nmd_after_check = pricing_nmd_after_check.reset_index(drop=True)
pricing_nmd_after_check.to_csv('pricing nmd.csv')

In [4]:
# ooening files
pricing_nmd_after_check = pd.read_csv('pricing nmd.csv')
pricing_crm_after_check = pd.read_csv('pricing crm.csv')

/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1324/2475118566.py:1: DtypeWarning: Columns (1,2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  pricing_nmd_after_check = pd.read_csv('pricing nmd.csv')


In [148]:
# merging bot pricing
# pricing_both_after_check = pd.concat([pricing_both_check, pricing_both__wothout_check])
# pricing_both_after_check = pricing_both_after_check.reset_index(drop=True)

In [149]:
# exporting to csv
#pricing_both_after_check.to_csv('pricinhg without removed.csv')

In [370]:
# opening csv
#pricing_both_after_check = pd.read_csv('pricinhg without removed.csv')

/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_6869/3405298310.py:2: DtypeWarning: Columns (1,2,11,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  pricing_both_after_check = pd.read_csv(f'{path}/pricinhg without removed.csv')


In [194]:
#######PRICING AND EXCEL-MERGING

In [208]:
# data types, removing 'US'
pricing_crm_after_check['SalesDeal'] = pricing_crm_after_check['SalesDeal'].astype(str)
pricing_crm_after_check['SalesDeal'] = pricing_crm_after_check['SalesDeal'].str.replace('US', '')
pricing_crm_after_check['SalesDeal'] = pricing_crm_after_check['SalesDeal'].astype(int)
pricing_nmd_after_check['SalesDeal'] = pricing_nmd_after_check['SalesDeal'].astype(str)
pricing_nmd_after_check['SalesDeal'] = pricing_nmd_after_check['SalesDeal'].str.replace('US', '')
pricing_nmd_after_check['SalesDeal'] = pricing_nmd_after_check['SalesDeal'].astype(int)

In [150]:
# pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].astype(str)
# pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].str.replace('US', '')
# pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].astype(int)


In [151]:
# data types


# excel_both['Sales Deal #'] = excel_both['Sales Deal #'].astype(str)
# pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].astype('int64')
# pricing_both_after_check['SalesDeal'] = pricing_both_after_check['SalesDeal'].astype(str)


In [209]:
# data types
excel_nmd['Sales Deal #'] = excel_nmd['Sales Deal #'].astype(str)
pricing_nmd_after_check['SalesDeal'] = pricing_nmd_after_check['SalesDeal'].astype('int64')
pricing_nmd_after_check['SalesDeal'] = pricing_nmd_after_check['SalesDeal'].astype(str)

excel_crm['Sales Deal #'] = excel_crm['Sales Deal #'].astype(str)
pricing_crm_after_check['SalesDeal'] = pricing_crm_after_check['SalesDeal'].astype('int64')
pricing_crm_after_check['SalesDeal'] = pricing_crm_after_check['SalesDeal'].astype(str)

In [210]:
# merge pricing and sales
excel_nmd['Customer Contract ID'] = pd.to_numeric(excel_nmd['Customer Contract ID'], errors='coerce')
pricing_nmd_after_check ['Customer'] = pd.to_numeric(pricing_nmd_after_check ['Customer'], errors='coerce')
excel_nmd['Sales Deal #'] = pd.to_numeric(excel_nmd['Sales Deal #'], errors='coerce')
pricing_nmd_after_check ['SalesDeal'] = pd.to_numeric(pricing_nmd_after_check['SalesDeal'], errors='coerce')
sales_nmd= pd.merge(pricing_nmd_after_check,excel_nmd, left_on='SalesDeal', right_on='Sales Deal #', how='inner')

# merge pricing and sales
excel_crm['Customer Contract ID'] = pd.to_numeric(excel_crm['Customer Contract ID'], errors='coerce')
pricing_crm_after_check ['Customer'] = pd.to_numeric(pricing_crm_after_check ['Customer'], errors='coerce')
excel_crm['Sales Deal #'] = pd.to_numeric(excel_crm['Sales Deal #'], errors='coerce')
pricing_crm_after_check ['SalesDeal'] = pd.to_numeric(pricing_crm_after_check['SalesDeal'], errors='coerce')
sales_crm= pd.merge(pricing_crm_after_check,excel_crm, left_on='SalesDeal', right_on='Sales Deal #', how='inner')

In [211]:
# merging sales, dropping duplicates
sales = pd.concat([sales_crm, sales_nmd])
sales = sales.drop_duplicates(subset = ['Product No',  'Price Program Name\n(free form text)','Customer Contract ID'],keep = 'first').reset_index(drop = True)

In [152]:
# # merge pricing and sales
# excel_both['Customer Contract ID'] = pd.to_numeric(excel_both['Customer Contract ID'], errors='coerce')
# pricing_both_after_check ['Customer'] = pd.to_numeric(pricing_both_after_check ['Customer'], errors='coerce')
# excel_both['Sales Deal #'] = pd.to_numeric(excel_both['Sales Deal #'], errors='coerce')
# pricing_both_after_check ['SalesDeal'] = pd.to_numeric(pricing_both_after_check['SalesDeal'], errors='coerce')

# sales= pd.merge(pricing_both_after_check,excel_both, left_on='SalesDeal', right_on='Sales Deal #', how='inner')

In [213]:
# aggregating  by contract id
sales_gr = sales.groupby([
 #   'SalesDeal',
'Customer Contract ID',
'Price Program Name\n(free form text)'
]).size().reset_index(name='counts')

In [214]:
#####XML######
# Getting numbers and list with products
list_all = []
Contract = tree.xpath('.//ContractOffer')
for contract in Contract:
    Program = contract.xpath('.//Program')
    for program in Program:
        Discount = program.xpath('.//DiscountOffListLI')
        Tiered = program.xpath('.//TieredLI')
        for discount in Discount:
            Price = discount.xpath('.//LIPrices')
            for price in Price:
                list_all.append([contract.attrib['Customer'], 
                    program.attrib['CustomerContractId'], program.attrib['SalesDealNo'], program.attrib['Name'],
                    discount.attrib['ProductNum'],
                    price.attrib['PriceHigh']])
    
        for discount in Tiered:
            Price = discount.xpath('.//TieredLIPrices')
            for price in Price:
                list_all.append([contract.attrib['Customer'], 
                program.attrib['CustomerContractId'], program.attrib['SalesDealNo'], program.attrib['Name'],
                discount.attrib['ProductNum'],
                price.attrib['PriceHigh']])

# dataframes
xml_all = pd.DataFrame(list_all)
xml_all.columns =['Customer ID', 'Contract ID','DealNo',
'Price Program Name', 
'Product ID', 'Price']

              


In [215]:
# data types
xml_all['Customer ID'] = xml_all['Customer ID'].str.extract('(\d+)', expand=False)
xml_all['Customer ID'] = xml_all['Customer ID'].astype(int)


In [216]:
# splitting lists of deals to separate rows
xml_all['DealNo']= xml_all['DealNo'].astype(str)
xml_all['DealNo']  = xml_all['DealNo'].apply(lambda x: x.split(','))
xml_all = xml_all.explode('DealNo')


In [1]:
# data types
#xml_all['DealNo'] = pd.to_numeric(xml_all['DealNo'])
#xml_all['DealNo'] = xml_all['DealNo'].astype(str)

In [218]:
# dropping duplicates
xml_all = xml_all.drop_duplicates(subset = ['Product ID', 'Price', 'Price Program Name','Customer ID'],keep = 'first').reset_index(drop = True)

In [97]:
# data types
# xml_all['Contract ID'] = pd.to_numeric(xml_all['Contract ID'])
# xml_all['Price'] = xml_all.Price.str.extract(r"(\d+\.\d+)")
# xml_all['Price'] = xml_all['Price'].astype(float)

In [219]:
# aggregating xml
xml_all_gr = xml_all.groupby([
   #  'DealNo', 
    'Contract ID', 'Price Program Name'
]).size().reset_index(name='counts')

In [220]:
# contract id
xml_all_gr['Contract ID'] = xml_all_gr['Contract ID'].str[:-4]

In [221]:
# Merging dataframes- counts
xml_all_gr['Contract ID'] = pd.to_numeric(xml_all_gr['Contract ID'])
sales_gr['Customer Contract ID'] = pd.to_numeric(sales_gr['Customer Contract ID'])
#xml_all_gr['DealNo'] = pd.to_numeric(xml_all_gr['DealNo'])
#sales_gr['SalesDeal'] = pd.to_numeric(sales_gr['SalesDeal'])

df_merged = pd.merge(sales_gr, xml_all_gr,  how='outer', left_on=['Customer Contract ID', 'Price Program Name\n(free form text)'], right_on = ['Contract ID', 'Price Program Name'])
df_merged.rename(columns = {'counts_x':'count from excel', 'counts_y':'count from xml'}, inplace = True)

In [222]:
# checking if the same
df_merged['Are the same?'] = ''
for index in tqdm(df_merged.index):
    if df_merged['count from xml'][index] == df_merged['count from excel'][index]:
        df_merged['Are the same?'][index] = 'yes'
    else:
        df_merged['Are the same?'][index] = 'no'
df_merged.to_excel('result counts_new4.xlsx')

  0%|          | 0/2646 [00:00<?, ?it/s]/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1988/879912546.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Are the same?'][index] = 'yes'
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1988/879912546.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Are the same?'][index] = 'no'
100%|██████████| 2646/2646 [00:00<00:00, 11687.33it/s]
